In [12]:
from src.util.io.io_util import IOUtil
from src.util.path.local_path import LocalPath
from src.util.mysql.dataframe_and_mysql import DataframeAndMysql

In [13]:
local_path = LocalPath()
file_path = local_path.get_key_folder() + "/prefect_trip_key.txt"

io_util = IOUtil()
database_info = io_util.read_key_file_data(file_path)

In [15]:
dataframe_and_mysql = DataframeAndMysql(database_name="perfect_trip_db", 
                                        username=database_info["username"], 
                                        password=database_info["password"], 
                                        hostname=database_info["hostname"], 
                                        port=database_info["port"])

In [16]:
dataframe_and_mysql.dataframe_from_mysql("user_master")

,user_id,username,password,first_name,last_name,nickname,tax_id,gender,user_group,phone_number,country,action,change_id,created_date,last_modified_date
0,2,test,e10adc3949ba59abbe56e057f20f883e,None,None,None,None,None,None,None,None,None,0,2024-07-28 11:58:22,2024-07-28 11:58:22
1,3,test1,e10adc3949ba59abbe56e057f20f883e,None,None,None,None,None,None,None,None,None,0,2024-07-28 12:00:42,2024-07-28 12:00:42
